APIs https://documenter.getpostman.com/view/664302/S1ENwy59

In [1]:
import requests
import folium
import math
from geopy.geocoders import Nominatim

Set Variables

In [8]:
city_name = "Amsterdam"
spthreshold=100
disthreshold=50


In [9]:
geolocator = Nominatim(user_agent="my_app")
location = geolocator.geocode(city_name)
if location:
    latitude = location.latitude
    longitude = location.longitude
    print(f"GPS Location of {city_name}: Latitude={latitude}, Longitude={longitude}")
else:
    print(f"Could not find GPS location for {city_name}")


GPS Location of Amsterdam: Latitude=52.3730796, Longitude=4.8924534


In [10]:
url=f'https://api.ebird.org/v2/ref/hotspot/geo?lat={latitude}&lng={longitude}&fmt=json&dist={disthreshold}'
content=requests.get(url)
sites=content.json()
len(sites)

446

In [11]:
filtered_sites = [obj for obj in sites if obj.get("numSpeciesAllTime",0) > spthreshold]
sorted_sites=sorted(filtered_sites, key=lambda obj: obj["numSpeciesAllTime"], reverse=True)
print(len(sorted_sites))
# sorted_sites

117


In [12]:
average_value = math.ceil(sum(site["numSpeciesAllTime"] for site in sorted_sites) / len(sorted_sites))
m = folium.Map(location=[latitude, longitude], zoom_start=6)
for site in sorted_sites:
    marker_color = "red" if site["numSpeciesAllTime"] > average_value else "orange"
    site_url = f"https://ebird.org/hotspot/{site['locId']}"
    popup_content = f"<a href='{site_url}' target='_blank'>{site['locName']}</a><br>{site['numSpeciesAllTime']}"
    folium.Marker(location=[site["lat"], site["lng"]], popup=popup_content,icon=folium.Icon(color=marker_color)).add_to(m)
legend_html = f"""
     <div style="position: fixed; 
                 bottom: 50px; left: 50px; width: 200px; height: 250px; 
                 border:2px solid grey; z-index:9999; font-size:14px;
                 background-color:white;
                 ">
         <p style="margin:10px;"><b>Legend</b></p>
         <p style="margin-left: 10px;">Radius: {disthreshold}</p>
         <p style="margin-left: 10px;">Number of sites: {len(sorted_sites)}</p>
         <p style="margin-left: 10px;">Avg Species: {average_value}</p>
         <p style="margin-left: 10px;"><img src="https://raw.githubusercontent.com/pointhi/leaflet-color-markers/master/img/marker-icon-red.png"> Higher than Average</p>
         <p style="margin-left: 10px;"><img src="https://raw.githubusercontent.com/pointhi/leaflet-color-markers/master/img/marker-icon-orange.png"> Lower than Average</p>
     </div>
     """

# Add the legend to the map
m.get_root().html.add_child(folium.Element(legend_html))

# Display the map
m.save("map.html")

In [7]:
m